In [8]:
import MapGrabber as MG
import glob
import numpy as np
import cv2
from scipy import ndimage
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def crop_k38():
    img_fps = glob.glob('.\\k38_lom_imgs\\*.gif')
    for fp in img_fps:
        img = ndimage.imread(fp)
        dst_img = cv2.cvtColor(MG.crop_border(img), cv2.COLOR_BGR2RGB)
        dst_fp = './k38_lom_cropped/' + fp.split('\\')[-1][:-4] + '.png'
        cv2.imwrite(dst_fp, dst_img)

In [9]:
sample_fps = ['./k38_lom_cropped/100k--k38-001.png', './k38_lom_cropped/100k--k38-002.png']

In [ ]:
# Algorithm outline
#------------------------
# 1. Pad one of the images
# 2. Scan the other image by next to it, taking the distance between the images at each step
# 3. Position the image at the place of minimum distance.
# 4. Stitch them together

In [104]:
len(imgL)

2187

In [105]:
len(imgR)

2183

In [129]:
colL = imgL[:, -1, :]
colR = imgR[:, 0, :]
padding = np.zeros([len(colL) - len(colR), 3])
filter_ = np.concatenate([padding, colR, padding])
colL = np.concatenate([padding, colL])
distances = []
for i in range(2*len(padding)):
    filter_ = np.roll(filter_, 1, axis=0)
    distances.append(abs(np.linalg.norm(filter_ - colL)))
shift = np.argmin(distances) + 1

In [131]:
from PIL import Image

def merge_images(file1, file2, shift):
    image1 = Image.open(file1)
    image2 = Image.open(file2)
    (width1, height1) = image1.size
    (width2, height2) = image2.size
    
    result_width = width1 + width2
    result_height = max(height1, height2)

    result = Image.new('RGB', (result_width, result_height))
    result.paste(im=image1, box=(0, 0))
    result.paste(im=image2, box=(width1, shift))
    return result

In [132]:
r = merge_images(sample_fps[0], sample_fps[1], shift)

In [133]:
r.save('sample_result.png')